# Большие каталоги
Не знаю пока, насколько будут полезны. Неудобно, что там потоки приведены в Янских

In [1]:
"""
from astroquery.vizier import Vizier

MALT90 = 'J/other/PASA/33.30/catalog'
ATLASGAL = 'J/A+A/568/A41/atl-csc'
MIPSGAL = '	J/AJ/149/64/catalog'
"""

"\nfrom astroquery.vizier import Vizier\n\nMALT90 = 'J/other/PASA/33.30/catalog'\nATLASGAL = 'J/A+A/568/A41/atl-csc'\nMIPSGAL = '\tJ/AJ/149/64/catalog'\n"

# Маленькие каталоги, содержащие данные об излучении в линиях молекул
## Краткая информация по каталогам:
1. Nobeyama molecular line survey of SCUBA-2 cores (Kim+, 2020). 	Properties of c-C3H2, DNC, HN13C, and N2D+ lines (229 rows) **229 источников**
2. Interstellar SiO sources (Harju+ 1998). Observed SiO J=2-1 line parameters, Onsala (188 rows) **188 источников**
3. Molecular lines in EGOs (Chen+, 2010). Observed HCO+(1-0) line properties of all 88 sample sources (119 rows) **88 источников**
4. RMS survey: molecular observations (Urquhart+, 2014) (1797 rows) **738 источников** 
5. SSEDIGISM-ATLASGAL in Galactic disc (Urquhart+, 2021). Fitted Gaussian parameters to CO spectra extracted towards ATLASGAL clumps 
6. The RMS survey: water masers of YSOs (Urquhart+, 2009). Peak maser flux (22.235GHz)
7. IRAM 3mm, 2mm & 1.3mm obs. of star-forming regions (Zhao+, 2023). Measured CH3OH spectra line parameters toward our sample by the IRAM 30m (1117 rows)
8. HOPS. III. Dense molecular gas properties (Longmore+, 2017) (687 rows)

In [1]:
from astroquery.vizier import Vizier
import pandas as pd

Vizier_cat = ['J/ApJS/249/33/table4',
              'J/ApJS/249/33/table1',
              'J/A+AS/132/211/onsa2-1',
              'J/ApJ/710/150/table3', 'J/MNRAS/437/1791/table1',
              'J/MNRAS/500/3050/table1', 'J/A+A/507/795/table2',
              'J/ApJS/266/29/table4']#, 'J/MNRAS/470/1462/tableb1']

cat_columns = [['Name', 'Tpkc','TpkDNC','TpkHN13C','TpkN2D'],
               ['Name', 'RAJ2000', 'DEJ2000', 'Dist'],
               ['Name', 'Dist', 'Line', 'TA_', 'FW'],
               ['Name', 'TdV', 'DelV', 'Tmb'],
               ['MSX', 'RAJ2000', 'DEJ2000', 'Trans', 'VLSR', 'Ta_', 'FWHM'],
               ['CSC', 'Trans', 'TA_', 'FWHM', '_RA', '_DE'],
               ['Field', 'RAJ2000', 'DEJ2000', 'Fpeak'],
               ['Name', 'Freq', 'Int', 'Tmb']]

data = []

for i in range(len(Vizier_cat)):
    data.append(pd.DataFrame())
    
for i in range(len(Vizier_cat)):
    for item in cat_columns[i]:
        data[i][item] = Vizier(row_limit=-1).get_catalogs(catalog=Vizier_cat[i])[0][item]

## Перевод из hms в deg

In [2]:
from astropy.coordinates import Angle
import astropy.units as u

def hms_to_deg(data):
    ra = []
    dec = []
    for i in range(len(data)):
        str = data['RAJ2000'][i].split()
        hms = ['h', 'm', 's']
        ra.append(Angle(str[0] + hms[0] + str[1] + hms[1] + str[2] + hms[2], unit=u.hourangle).deg)
    data['RAJ2000'] = ra
    for i in range(len(data)):
        str = data['DEJ2000'][i].split()
        hms = ['d', 'm', 's']
        dec.append(Angle(str[0] + hms[0] + str[1] + hms[1] + str[2] + hms[2], unit=u.deg).deg)
    data['DEJ2000'] = dec

# Причешем каталог №1 (Nobeyama molecular line survey of SCUBA-2 cores)

In [3]:
data[0] = data[0].groupby('Name').mean()
data[0] = data[0].merge(data[1], how = 'inner', on='Name')

In [4]:
hms_to_deg(data[0])

In [5]:
data[0]

,Name,Tpkc,TpkDNC,TpkHN13C,TpkN2D,RAJ2000,DEJ2000,Dist
0,G190.15-13.75North,0.13,0.200,0.15,0.210,79.754625,13.134944,0.38
1,G190.15-13.75South,0.13,0.180,0.14,0.180,79.668958,13.028222,0.38
2,G191.90-11.21North,0.12,0.160,0.13,0.140,82.870792,12.981944,0.38
3,G191.90-11.21South,0.38,0.440,0.11,0.160,82.882208,12.937500,0.38
4,G192.12-10.90North,0.77,0.520,0.18,0.190,83.261000,12.964889,0.38
...,...,...,...,...,...,...,...,...
202,SCOPEG195.71-02.32,0.36,0.450,0.11,0.130,92.710000,14.171000,2.05
203,SCOPEG195.74-02.30,0.41,0.710,0.16,0.160,92.743000,14.158000,2.05
204,SCOPEG202.30+02.53,0.19,0.300,0.13,0.150,100.243000,10.615000,0.76
205,SCOPEG202.31+02.52,0.17,0.250,0.10,0.110,100.247000,10.602000,0.76


# Причешем каталог №4 (RMS survey: molecular observations)

In [7]:
hms_to_deg(data[4])

In [8]:
data[4]

,MSX,RAJ2000,DEJ2000,Trans,VLSR,Ta_,FWHM
0,G010.1615-00.3623,272.362000,-20.324500,"NH3(1,1)",13.020000,0.41,13.76
1,G010.1880-00.3162,272.332583,-20.279000,"NH3(1,1)",10.870000,0.38,12.09
2,G010.3844+02.2128,270.094500,-18.868889,"NH3(1,1)",5.390000,0.10,1.43
3,G010.4413+00.0101,272.159292,-19.899278,"NH3(1,1)",66.540001,0.29,14.42
4,G010.5067+02.2285,270.144083,-18.754889,"NH3(1,1)",21.290001,0.58,3.58
...,...,...,...,...,...,...,...
1792,G349.1242+00.0240,259.209167,-38.017389,13CO(J=1-0),-78.110001,0.45,3.97
1793,G349.1242+00.0240,259.209167,-38.017389,13CO(J=1-0),-92.790001,0.37,7.30
1794,G349.1772+00.0277,259.244292,-37.971917,13CO(J=1-0),15.440000,2.30,5.89
1795,G349.1772+00.0277,259.244292,-37.971917,13CO(J=1-0),-78.349998,1.04,5.13


In [10]:
import numpy as np

unique_name = data[4]['MSX'].unique() # Список уникальных объектов в каталоге
unique_trans = data[4]['Trans'].unique() # Список уникальных переходов в каталоге

buff = np.zeros((len(unique_name), 14))
names = [] 
coords = []
n = 0 # уникальные объекты

def fill_data(n, j, buff, data):
    buff[n][j*3] = data[4]['VLSR'][i]
    buff[n][j*3 + 1] = data[4]['Ta_'][i]
    buff[n][j*3 + 2] = data[4]['FWHM'][i]
    
for i in range(len(data[4])):
    if i==0:
        names.insert(n, data[4]['MSX'][i])
        coords.insert(n, [data[4]['RAJ2000'][i], data[4]['DEJ2000'][i]])
        for j in range(len(unique_trans)):  
                if data[4]['Trans'][i] == unique_trans[j]:
                    fill_data(n, j, buff, data)
                else:
                    continue
    else:
        repeat = False
        for k in range(n + 1):
            if data[4]['MSX'][i] == names[k]:
                repeat = True
                for j in range(len(unique_trans)):
                    if data[4]['Trans'][i] == unique_trans[j]:
                        fill_data(k, j, buff, data)
                    else:
                        continue
            else:
                continue
        if repeat == True:
            continue
        else:
            n += 1
            names.insert(n, data[4]['MSX'][i])
            coords.insert(n, [data[4]['RAJ2000'][i], data[4]['DEJ2000'][i]])
            for j in range(len(unique_trans)):  
                if data[4]['Trans'][i] == unique_trans[j]:
                    fill_data(n, j, buff, data)
                else:
                    continue
data4 = np.zeros((len(unique_name), 14))
for i in range(len(unique_name)):
    data4[i][0] = coords[i][0]
    data4[i][1] = coords[i][1]
    for j in range(12):
        data4[i][j+2] = buff[i][j]


In [11]:
columns = ['RAJ2000', 'DEJ2000',
          'NH3(1,1) VLSR', 'NH3(1,1) Ta_', 'NH3(1,1) FWHM',
          'NH3(2,2) VLSR', 'NH3(2,2) Ta_', 'NH3(2,2) FWHM',
          'CS(J=2-1) VLSR', 'CS(J=2-1) Ta_', 'CS(J=2-1) FWHM',
          '13CO(J=1-0) VLSR', '13CO(J=1-0) Ta_', '13CO(J=1-0) FWHM']
data4_df = pd.DataFrame(data4, columns=columns)
data4_df.insert(0, 'MSX', pd.DataFrame(names))
data[4] = data4_df

In [12]:
data[4]

,MSX,RAJ2000,DEJ2000,"NH3(1,1) VLSR","NH3(1,1) Ta_","NH3(1,1) FWHM","NH3(2,2) VLSR","NH3(2,2) Ta_","NH3(2,2) FWHM",CS(J=2-1) VLSR,CS(J=2-1) Ta_,CS(J=2-1) FWHM,13CO(J=1-0) VLSR,13CO(J=1-0) Ta_,13CO(J=1-0) FWHM
0,G010.1615-00.3623,272.362000,-20.324500,13.020000,0.41,13.76,12.860000,0.19,6.71,0.0,0.0,0.0,0.000000,0.00,0.00
1,G010.1880-00.3162,272.332583,-20.279000,10.870000,0.38,12.09,10.910000,0.17,6.26,0.0,0.0,0.0,0.000000,0.00,0.00
2,G010.3844+02.2128,270.094500,-18.868889,5.390000,0.10,1.43,NaN,NaN,NaN,0.0,0.0,0.0,0.000000,0.00,0.00
3,G010.4413+00.0101,272.159292,-19.899278,66.540001,0.29,14.42,66.330002,0.05,3.48,0.0,0.0,0.0,0.000000,0.00,0.00
4,G010.5067+02.2285,270.144083,-18.754889,21.290001,0.58,3.58,21.340000,0.11,1.94,0.0,0.0,0.0,0.000000,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,G347.8944-00.1713,258.497625,-39.130556,0.000000,0.00,0.00,0.000000,0.00,0.00,0.0,0.0,0.0,-92.540001,0.99,4.32
734,G347.9023+00.0481,258.274208,-38.995778,0.000000,0.00,0.00,0.000000,0.00,0.00,0.0,0.0,0.0,-97.959999,0.34,-1.38
735,G348.6491+00.0225,258.860125,-38.404889,0.000000,0.00,0.00,0.000000,0.00,0.00,0.0,0.0,0.0,-72.769997,2.83,5.60
736,G349.0631+00.1078,259.078750,-38.018333,0.000000,0.00,0.00,0.000000,0.00,0.00,0.0,0.0,0.0,-78.070000,1.71,5.60


# Попробуем пересечь каталог 1 и каталог 4

# Кластеризация

In [7]:
from sklearn import preprocessing
import numpy as np


In [78]:
"""
from sklearn.cluster import HDBSCAN

hdb = HDBSCAN(algorithm='auto', metric='euclidean')
hdb.fit(data_np)
"""

"\nfrom sklearn.cluster import HDBSCAN\n\nhdb = HDBSCAN(algorithm='auto', metric='euclidean')\nhdb.fit(data_np)\n"

In [76]:
#hdb.labels_

In [75]:
"""
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

Nobeyama_data['label'] = pd.DataFrame(hdb.labels_)
sns.pairplot(Nobeyama_data, corner=True, hue='label', palette='tab10') # colorblind Set1
plt.rcParams['font.size'] = '21'
#plt.subplots_adjust(left=0.001)
plt.tight_layout()
"""

"\n%matplotlib inline\nimport matplotlib.pyplot as plt\nimport seaborn as sns\n\nNobeyama_data['label'] = pd.DataFrame(hdb.labels_)\nsns.pairplot(Nobeyama_data, corner=True, hue='label', palette='tab10') # colorblind Set1\nplt.rcParams['font.size'] = '21'\n#plt.subplots_adjust(left=0.001)\nplt.tight_layout()\n"